In [1]:
import numpy as np
import pandas as pd
import pickle
import requests
import torch.nn as nn
from torch.utils.data import DataLoader
import json
import sparql
import ast

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sys


In [2]:
def execute_sparql_wikidata(url,query):
    r = requests.get(url, params = {'format': 'json', 'query': query})
    return r.json()

In [3]:
q = ('''SELECT DISTINCT ?station, ?orbits WHERE { ?station a <http://dbpedia.org/ontology/SpaceStation> . 
    ?station <http://dbpedia.org/property/orbits> ?orbits . 
     FILTER(?orbits > 50000) } ORDER BY DESC(?orbits)''')
result = sparql.query('http://dbpedia.org/sparql', q)

In [4]:
result.variables

['station', 'orbits']

In [5]:
path = "D:\\.Memoria\\Test\\Datasets\\"
csv_name = "ds_trainval_pred_filtered.csv"
df = pd.read_csv(path+csv_name, delimiter="ᶶ", engine='python', encoding='utf-8')
df_reducido = df[df['time'] <=65]
columns = df_reducido.columns
print(columns)
print(df.shape)
print(df_reducido.shape)



Index(['id', 'query', 'time', 'assign', 'bgp', 'distinct', 'extend', 'filter',
       'filter_bound', 'filter_contains', 'filter_eq', 'filter_exists',
       'filter_ge', 'filter_gt', 'filter_isBlank', 'filter_isIRI',
       'filter_isLiteral', 'filter_lang', 'filter_langMatches', 'filter_le',
       'filter_lt', 'filter_ne', 'filter_not', 'filter_notexists', 'filter_or',
       'filter_regex', 'filter_sameTerm', 'filter_str', 'filter_strends',
       'filter_strstarts', 'filter_subtract', 'graph', 'group', 'has_slice',
       'join', 'json_cardinality', 'leftjoin', 'max_slice_limit',
       'max_slice_start', 'minus', 'multi', 'notoneof', 'order', 'path*',
       'path+', 'path?', 'pathN*', 'pathN+', 'pcs0', 'pcs1', 'pcs10', 'pcs11',
       'pcs12', 'pcs13', 'pcs14', 'pcs15', 'pcs16', 'pcs17', 'pcs18', 'pcs19',
       'pcs2', 'pcs20', 'pcs21', 'pcs22', 'pcs23', 'pcs24', 'pcs3', 'pcs4',
       'pcs5', 'pcs6', 'pcs7', 'pcs8', 'pcs9', 'project', 'reduced',
       'sequence', 'slice', 'to

In [6]:
max_index = df_reducido["treesize"]
for i in range(0,len(max_index)+1):
    print(i,max_index[i])

0 8.0
1 2.0
2 2.0
3 2.0
4 2.0
5 2.0
6 5.0
7 2.0
8 2.0
9 8.0
10 2.0
11 2.0
12 8.0
13 2.0
14 8.0
15 2.0
16 8.0
17 2.0
18 3.0
19 8.0
20 2.0
21 8.0
22 3.0
23 2.0
24 2.0
25 3.0
26 8.0
27 2.0
28 1.0
29 6.0
30 2.0
31 4.0
32 2.0
33 2.0
34 2.0
35 2.0
36 3.0
37 2.0
38 2.0
39 2.0
40 2.0
41 3.0
42 3.0
43 8.0
44 8.0
45 2.0
46 8.0
47 1.0
48 2.0
49 8.0
50 8.0
51 5.0
52 2.0
53 2.0
54 3.0
55 2.0
56 8.0
57 2.0
58 2.0
59 6.0
60 8.0
61 2.0
62 2.0
63 8.0
64 2.0
65 2.0
66 8.0
67 2.0
68 8.0
69 8.0
70 2.0
71 2.0
72 8.0
73 2.0
74 8.0
75 2.0
76 2.0
77 2.0
78 3.0
79 8.0
80 1.0
81 2.0
82 3.0
83 2.0
84 2.0
85 8.0
86 2.0
87 1.0
88 2.0
89 2.0
90 3.0
91 2.0
92 2.0
93 8.0
94 8.0
95 8.0
96 2.0
97 3.0
98 5.0
99 2.0
100 2.0
101 2.0
102 2.0
103 8.0
104 2.0
105 3.0
106 2.0
107 8.0
108 2.0
109 2.0
110 2.0
111 2.0
112 2.0
113 2.0
114 2.0
115 3.0
116 1.0
117 8.0
118 3.0
119 1.0
120 2.0
121 2.0
122 2.0
123 6.0
124 8.0
125 3.0
126 2.0
127 2.0
128 3.0
129 8.0
130 8.0
131 2.0
132 2.0
133 2.0
134 1.0
135 8.0
136 8.0
137 2.0
138 3.

1271 8.0
1272 2.0
1273 3.0
1274 2.0
1275 8.0
1276 2.0
1277 8.0
1278 3.0
1279 2.0
1280 2.0
1281 4.0
1282 2.0
1283 2.0
1284 2.0
1285 2.0
1286 3.0
1287 2.0
1288 8.0
1289 2.0
1290 2.0
1291 2.0
1292 2.0
1293 2.0
1294 4.0
1295 2.0
1296 8.0
1297 2.0
1298 8.0
1299 8.0
1300 2.0
1301 8.0
1302 8.0
1303 8.0
1304 2.0
1305 2.0
1306 2.0
1307 8.0
1308 4.0
1309 2.0
1310 5.0
1311 2.0
1312 2.0
1313 2.0
1314 3.0
1315 5.0
1316 8.0
1317 8.0
1318 2.0
1319 2.0
1320 2.0
1321 2.0
1322 2.0
1323 2.0
1324 2.0
1325 3.0
1326 8.0
1327 2.0
1328 3.0
1329 2.0
1330 2.0
1331 2.0
1332 2.0
1333 8.0
1334 2.0
1335 3.0
1336 2.0
1337 8.0
1338 3.0
1339 8.0
1340 3.0
1341 2.0
1342 2.0
1343 3.0
1344 2.0
1345 8.0
1346 2.0
1347 4.0
1348 2.0
1349 8.0
1350 2.0
1351 2.0
1352 7.0
1353 6.0
1354 3.0
1355 2.0
1356 8.0
1357 2.0
1358 2.0
1359 3.0
1360 2.0
1361 2.0
1362 2.0
1363 3.0
1364 8.0
1365 2.0
1366 2.0
1367 2.0
1368 2.0
1369  3.0
1464 2.0
1465 2.0
1466 2.0
1467 2.0
1468 3.0
1469 2.0
1470 3.0
1471 2.0
1472 8.0
1473 2.0
1474 8.0
1475 1.0


2847 2.0
2848 2.0
2849 8.0
2850 8.0
2851 3.0
2852 3.0
2853 3.0
2854 8.0
2855 2.0
2856 2.0
2857 8.0
2858 8.0
2859 3.0
2860 2.0
2861 2.0
2862 2.0
2863 2.0
2864 2.0
2865 2.0
2866 8.0
2867 1.0
2868 8.0
2869 2.0
2870 8.0
2871 2.0
2872 4.0
2873 2.0
2874 2.0
2875 2.0
2876 2.0
2877 2.0
2878 2.0
2879 1.0
2880 2.0
2881 2.0
2882 2.0
2883 2.0
2884 6.0
2885 2.0
2886 8.0
2887 2.0
2888 8.0
2889 8.0
2890 2.0
2891 1.0
2892 3.0
2893 2.0
2894 2.0
2895 2.0
2896 2.0
2897 2.0
2898 2.0
2899 2.0
2900 2.0
2901 3.0
2902 2.0
2903 2.0
2904 2.0
2905 8.0
2906 2.0
2907 8.0
2908 2.0
2909 2.0
2910 3.03007 3.0
3008 2.0
3009 1.0
3010 3.0
3011 2.0
3012 8.0
3013 2.0
3014 8.0
3015 2.0
3016 8.0
3017 2.0
3018 8.0
3019 8.0
3020 3.0
3021 2.0
3022 2.0
3023 8.0
3024 2.0
3025 2.0
3026 2.0
3027 2.0
3028 2.0
3029 3.0
3030 8.0
3031 2.0
3032 2.0
3033 8.0
3034 2.0
3035 3.0
3036 1.0
3037 2.0
3038 2.0
3039 3.0
3040 3.0
3041 8.0
3042 2.0
3043 2.0
3044 2.0
3045 2.0
3046 2.0
3047 3.0
3048 8.0
3049 4.0
3050 2.0
3051 8.0
3052 3.0
3053 1.0
30

4393 3.0
4394 5.0
4395 2.0
4396 3.0
4397 3.0
4398 2.0
4399 8.0
4400 2.0
4401 5.0
4402 2.0
4403 3.0
4404 2.0
4405 2.0
4406 8.0
4407 4.0
4408 2.0
4409 2.0
4410 5.0
4411 8.0
4412 3.0
4413 4.0
4414 8.0
4415 2.0
4416 7.0
4417 5.0
4418 3.0
4419 3.0
4420 2.0
4421 5.0
4422 
4516 2.0
4517 8.0
4518 8.0
4519 3.0
4520 8.0
4521 5.0
4522 8.0
4523 8.0
4524 4.0
4525 3.0
4526 2.0
4527 2.0
4528 3.0
4529 5.0
4530 8.0
4531 3.0
4532 8.0
4533 5.0
4534 3.0
4535 5.0
4536 8.0
4537 2.0
4538 2.0
4539 3.0
4540 8.0
4541 6.0
4542 8.0
4543 2.0
4544 5.0
4545 4.0
4546 2.0
4547 8.0
4548 4.0
4549 8.0
4550 8.0
4551 2.0
4552 8.0
4553 2.0
4554 8.0
4555 2.0
4556 2.0
4557 2.0
4558 2.0
4559 1.0
4560 4.0
4561 8.0
4562 8.0
4563 7.0
4564 2.0
4565 1.0
4566 1.0
4567 1.0
4568 3.0
4569 8.0
4570 2.0
4571 4.0
4572 8.0
4573 2.0
4574 2.0
4575 4.0
4576 2.0
4577 4.0
4578 2.0
4579 5.0
4580 2.0
4581 2.0
4582 2.0
4583 5.0
4584 8.0
4585 5.0
4586 3.0
4587 3.0
4588 2.0
4589 8.0
4590 3.0
4591 8.0
4592 2.0
4593 2.0
4594 2.0
4595 2.0
4596 8.0
4597

6023 2.0
6024 5.0
6025 8.0
6026 8.0
6027 3.0
6028 4.0
6029 2.0
6030 8.0
6031 3.0
6032 8.0
6033 8.0
6034 3.0
6035 1.0
6036 2.0
6037 8.0
6038 2.0
6039 3.0
6040 8.0
6041 8.0
6042 5.0
6043 1.0
6044 8.0
6045 2.0
6046 3.0
6047 1.0
6048 3.0
6049 2.0
6050 8.0
6051 2.0
6052 2.0
6053 1.0
6054 2.0
6055 3.0
6056 2.0
6057 5.0
6058 2.0
6059 2.0
6060 8.0
6061 5.0
6062 6.0
6063 5.0
6064 2.0
6065 8.0
6066 3.0
6067 2.0
6068 2.0
6069 6.0
6070 8.0
6071 2.0
6072 8.0
6073 3.0
6074 2.0
6075 3.0
6076 8.0
6077 2.0
6078 8.0
6079 2.0
6080 5.0
6081 3.0
6082 2.0
6083 5.0
6084 2.0
6085 2.0
6086 2.0
6087 2.0
6088 8.0
6089 4.0
6090 2.0
6091 2.0
6092 3.0
6093 1.0
6094 4.0
6095 4.0
6096 3.0
6097 2.0
6098 2.0
6099 4.0
6100 6.0
6101 8.0
6102 3.0
6103 8.0
6104 2.0
6105 5.0
6106 2.0
6107 3.0
6108 2.0
6109 8.0
6110 3.0
6111 5.0
6112 8.0
6113 2.0
6114 2.0
6115 2.0
6116 1.0
6117 2.0
6118 2.0
6119 3.0
6120 2.0
6121 4.0
6122 4.0
6123 8.0
6124 3.0
6125 5.0
6126 2.0
6127 8.0
6128 2.0
6129 2.0
6130 1.0
6131 8.0
6132 2.0
6133 2.0
6

...

16355 4.0
16356 4.0
16357 4.0
16358 2.0
16359 4.0
16360 4.0
16361 4.0
16362 4.0
16363 4.0
16364 4.0
16365 4.0
16366 4.0
16367 4.0
16368 1.0
16369 6.0
16370 4.0
16371 5.0
16372 2.0
16373 4.0
16374 4.0
16375 4.0
16376 4.0
16377 4.0
16378 4.0
16379 4.0
16380 4.0
16381 3.0
16382 4.0
16383 4.0
16384 4.0
16385 4.0
16386 2.0
16387 6.0
16388 4.0
16389 2.0
16390 4.0
16391 4.0
16392 2.0
16393 4.0
16394 6.0
16395 7.0
16396 4.0
16397 4.0
16398 4.0
16399 4.0
16400 4.0
16401 4.0
16402 4.0
16403 2.0
16404 3.0
16405 4.0
16406 4.0
16407 6.0
16408 4.0
 4.0
16494 4.0
16495 4.0
16496 4.0
16497 4.0
16498 4.0
16499 4.0
16500 4.0
16501 4.0
16502 4.0
16503 4.0
16504 4.0
16505 2.0
16506 3.0
16507 4.0
16508 4.0
16509 4.0
16510 4.0
16511 4.0
16512 4.0
16513 4.0
16514 1.0
16515 5.0
16516 3.0
16517 4.0
16518 4.0
16519 4.0
16520 4.0
16521 4.0
16522 4.0
16523 2.0
16524 4.0
16525 4.0
16526 4.0
16527 6.0
16528 1.0
16529 5.0
16530 4.0
16531 4.0
16532 4.0
16533 4.0
16534 4.0
16535 4.0
16536 4.0
16537 4.0
16538 4.0
16539

17792 4.0
17793 3.0
17794 4.0
17795 4.0
17796 3.0
17797 4.0
17798 3.0
17799 4.0
17800 4.0
17801 4.0
17802 4.0
17803 4.0
17804 4.0
17805 4.0
17806 4.0
17807 1.0
17808 4.0
17809 4.0
17810 4.0
17811 4.0
17812 2.0
17813 4.0
17814 4.0
17815 4.0
17816 4.0
17817 4.0
17818 4.0
17819 4.0
17820 4.0
17821 4.0
17822 4.0
17823 4.0
17824 4.0
17825 4.0
17826 4.0
17827 4.0
17828 4.0
17829 4.0
17830 4.0
17831 2.0
17832 8.0
17833 4.0
17834 5.0
17835 7.0
17836 4.0
17837 4.0
17838 4.0
17839 4.0
17840 4.0
17841 4.0
17842 3.0
17843 5.0
17844 4.0
17845 4.0
17846 4.0
17847 4.0
17848 4.0
17849 4.0
17850 5.0
17851 4.0
17852 4.0
17853 5.0
17854 4.0
17855 4.0
17856 4.0
17857 4.0
17858 4.0
17859 4.0
17860 4.0
17861 4.0
17862 4.0
17863 4.0
17864 4.0
17865 2.0
17866 8.0
17867 4.0
17868 2.0
17869 4.0
17870 4.0
17871 5.0
17872 4.0
17873 4.0
17874 3.0
17875 4.0
17876 4.0
17877 4.0
17878 1.0
17879 2.0
17880 2.0
17881 4.0
17882 8.0
17883 4.0
17884 4.0
17885 4.0
17886 2.0
17887 4.0
17888 4.0
17889 4.0
17890 4.0
17891 4.0


19085 4.0
19086 4.0
19087 4.0
19088 4.0
19089 2.0
19090 4.0
19091 4.0
19092 5.0
19093 3.0
19094 4.0
19095 4.0
19096 4.0
19097 4.0
19098 2.0
19099 2.0
19100 4.0
19101 4.0
19102 4.0
19103 4.0
19104 4.0
19105 4.0
19106 2.0
19107 4.0
19108 4.0
19109 7.0
19110 4.0
19111 4.0
19112 4.0
19113 4.0
19114 4.0
19115 4.0
19116 4.0
19117 4.0
19118 4.0
19119 4.0
19120 4.0
19121 4.0
19122 4.0
19123 4.0
19124 4.0
19125 4.0
19126 
19206 4.0
19207 4.0
19208 4.0
19209 4.0
19210 4.0
19211 4.0
19212 2.0
19213 4.0
19214 4.0
19215 4.0
19216 4.0
19217 4.0
19218 3.0
19219 4.0
19220 4.0
19221 4.0
19222 4.0
19223 4.0
19224 3.0
19225 4.0
19226 4.0
19227 4.0
19228 4.0
19229 4.0
19230 4.0
19231 3.0
19232 4.0
19233 4.0
19234 4.0
19235 4.0
19236 4.0
19237 4.0
19238 4.0
19239 4.0
19240 4.0
19241 4.0
19242 4.0
19243 4.0
19244 4.0
19245 4.0
19246 4.0
19247 4.0
19248 4.0
19249 4.0
19250 1.0
19251 4.0
19252 4.0
19253 2.0
19254 4.0
19255 4.0
19256 4.0
19257 4.0
19258 4.0
19259 4.0
19260 4.0
19261 4.0
19262 4.0
19263 5.0
192

20654 4.0
20655 4.0
20656 4.0
20657 4.0
20658 4.0
20659 4.0
20660 4.0
20661 4.0
20662 1.0
20663 4.0
20664 4.0
20665 4.0
20666 4.0
20667 4.0
20668 4.0
20669 4.0
20670 4.0
20671 4.0
20672 4.0
20673 4.0
20674 4.0
20675 3.0
20676 4.0
20677 4.0
20678 4.0
20679 4.0
20680 4.0
20681 4.0
20682 4.0
20683 4.0
20684 4.0
20685 4.0
20686 4.0
20687 4.0
20688 4.0
20689 4.0
20690 4.0
20691 4.0
20692 4.0
20693 4.0
20694 4.0
20695 4.0
20696 4.0
20697 4.0
20698 4.0
20699 5.0
20700 4.0
20701 4.0
20702 4.0
20703 4.0
20704 4.0
20705 4.0
20706 4.0
20707 4.0
20708 4.0
20709 4.0
20710 4.0
20711 4.0
20712 4.0
20713 4.0
20714 4.0
20715 4.0
20716 4.0
20717 4.0
20718 4.0
20719 4.0
20720 4.0
20721 4.0
20722 4.0
20723 4.0
20724 4.0
20725 4.0
20726 4.0
20727 4.0
20728 4.0
20729 4.0
20730 3.0
20731 4.0
20732 4.0
20733 4.0
20734 4.0
20735 4.0
20736 4.0
20737 4.0
20738 4.0
20739 4.0
20740 4.0
20741 4.0
20742 4.0
20743 4.0
20744 4.0
20745 4.0
20746 4.0
20747 4.0
20748 4.0
20749 2.0
20750 4.0
20751 4.04.0
20911 4.0
20912 4

KeyError: 21235

In [ ]:
pd.set_option('display.max_rows', None)
df_reducido[['tree_tdb']]

In [ ]:
pd.set_option('display.max_rows', 10)

##### Observaciones
* El treesize pareciera tener q ver con los valores iniciales del tree_tdb
* En la columna trees, los nodos JOIN y los nodos hojas "VAR_URI_VAR,VAR_URI_URI" (los nodos SCAN), etc.. van acompañados por los predicados que involucran (los valores entre medio del triple)
* El formato que tanto trees como tree_tdb se lee de la siguiente forma [raiz[hijo1,hijo2]]
* Nodos como distinc, project, filter, etc... va relacionado con las consultas (distinc = DISTINC, project = SELECT algo, filter = FILTER, etc...


In [ ]:
lista_trees = json.loads(df_reducido['trees'][test_id])
print(type(lista_trees))
print(lista_trees)
print("---------")
for i in lista_trees:
    print("ooooooo")
    print (i)
    print("ooooooo")

In [ ]:
def binaryTreeTDBFromStringToList(string):
    returnList = []
    convert = ast.literal_eval(string)
    
    

In [ ]:

lista_tree_tdb = df_reducido['tree_tdb'][test_id]
print(type(lista_tree_tdb))
print(lista_tree_tdb)
print(len(lista_tree_tdb))
print("---------")
a = ast.literal_eval(lista_tree_tdb)
print(a)
print(len(a))
print(a[0])
print(a[1])
print(type(a[0]))
print(type(a[1]))
for i in a[1]:
    print(i)
    

In [ ]:
'''
url = 'https://query.wikidata.org/sparql'
## http://www.sparql.org/query-validator.html
test1 = 
SELECT  ?var1 ?var2
WHERE
  { ?var1  <http://www.wikidata.org/prop/direct/P31>  <http://www.wikidata.org/entity/Q4022> .
    ?var2  <http://schema.org/about>  ?var1 ;
           <http://schema.org/inLanguage>  "fa"
  }

test2 = df_reducido['query'][test_id]
result1 = sparql.query(url,test1)
result2 = sparql.query(url,test2)
print(test1)
print(test2)
print("                      ")
print(result1.fetchall())
print(result2.fetchall())
print("                      ")
print(len(result1.fetchall())==len(result2.fetchall()))
print(result1.fetchall()==result2.fetchall())
'''